<a href="https://colab.research.google.com/github/Matteo-Candi/ADM_HW3/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 3 - Places of the world**
---

## **1. Data collection**

### 1.1. Get the list of places

### 1.2. Crawl places

### 1.3 Parse downloaded pages

---
## **2. Search Engine**

### 2.1. Conjunctive query

 2.1.1) Create your index!

2.1.2) Execute the query

### 2.2. Conjunctive query & Ranking score

2.2.1) Inverted index

2.2.2) Execute the query

---
## **3. Define a new score!**

---
## **4. Visualizing the most relevant places**

---
## **5. BONUS: More complex search engine**

---
## **6. Command line question**

---
## **7. Theoretical question**